***
## notebook config

In [1]:
ON_COLAB = False
ASSESS = True

In [2]:
if ON_COLAB:
    from google.colab import drive
    from google.colab import files
    drive.mount('/content/drive', force_remount=True)
    
    !pip install --upgrade kaggle > /dev/null 2>&1
    !mkdir -p ~/.kaggle/ && cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
    
    !free -h
    
    !pip install --upgrade category_encoders > /dev/null 2>&1
    !pip install --upgrade tsforest > /dev/null 2>&1
    !pip install --upgrade lightgbm > /dev/null 2>&1
    !pip install --upgrade optuna > /dev/null 2>&1

In [3]:
if ON_COLAB:
    !kaggle datasets download -d mavillan/meli-2021 --force --unzip
    !ls -halt
    input_path = "./"
    print("input_path:", input_path)
    subs_path = "/content/drive/MyDrive/meli2021/subs"
    print("subs_path:", subs_path)
    results_path = "/content/drive/MyDrive/meli2021/results"
    print("results_path:", results_path)
else:
    input_path = "../data"
    print("input_path:", input_path)
    subs_path = "../subs"
    print("subs_path:", subs_path)
    results_path = "../results"
    print("results_path:", results_path)

input_path: ../data
subs_path: ../subs
results_path: ../results


***

In [4]:
from copy import deepcopy
import gc
import dill
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import time
from tqdm import tqdm
import yaml
import shap
import optuna

from sklearn import preprocessing
import category_encoders as ce
import lightgbm as lgb
from tsforest.forecast import LightGBMForecaster

import sys
sys.path.append("../utils")
from memory import reduce_mem_usage
from metrics import RMSSE, ranked_probability_score, scoring_function, binarize_predictions
from inventory import InventoryDaysPredictor
from features import Featurador

In [5]:
SEED = 19
np.random.seed(SEED)
pd.set_option('display.max_columns', None)

***
## Data preparation

In [6]:
with open(f"../data/skus_assess_m1.yaml", "r") as file:
    skus_assess_m1 = yaml.load(file, Loader=yaml.FullLoader)
    print(f"len(skus_assess_m1): {len(skus_assess_m1)}")
    file.close()

with open(f"../data/skus_assess_m2.yaml", "r") as file:
    skus_assess_m2 = yaml.load(file, Loader=yaml.FullLoader)
    print(f"len(skus_assess_m2): {len(skus_assess_m2)}")
    file.close()
    
skus_for_test = pd.read_csv(f"{input_path}/test_data.csv").sku.values
print(f"len(skus_for_test): {len(skus_for_test)}")

len(skus_assess_m1): 482635
len(skus_assess_m2): 272130
len(skus_for_test): 551472


In [7]:
if ASSESS:
    scales = pd.read_csv(f"{input_path}/scales.csv")
    
    #skus_assess_m1 = set(skus_assess_m1) & set(skus_for_test)
    #print("# skus assess & test:", len(skus_assess_m1))

    dataset = (
        pd.read_parquet(f"{input_path}/train-m1.parquet")
        .query("sku in @skus_assess_m1")
        .rename({"date":"ds", "sold_quantity":"y"}, axis=1,)
        .sort_values(["sku","ds"])
        .reset_index(drop=True)
    )
    
    with open("../encoders/encoder-stg1.dill", "rb") as file:
        encoder = dill.load(file)
        file.close()
    
    transformed = encoder.transform(dataset[encoder.cols].astype("category"))
    for col in transformed.columns:
        dataset[col+"_glm"] = transformed[col].values

In [8]:
if ASSESS:
    raw = pd.read_parquet(f"{input_path}/train_data.parquet", columns=["sku", "date", "sold_quantity", "minutes_active"])
    metadata = pd.read_csv(f"{input_path}/metadata.csv", usecols=["sku","item_domain_id","site_id"])
    raw = pd.merge(raw, metadata, how="inner", on="sku")

    # compute q_mean and q_std features (w/o leakage)
    feat = Featurador(raw)
    feat.fit(left_limit="2021-02-01", right_limit="2021-03-01")
    dataset = feat.transform(dataset)

***
## model config

In [9]:
categorical_features = {
    "currency": "default",
    "listing_type": "default",
    "shipping_logistic_type": "default",
    "shipping_payment": "default",
    "item_domain_id": "default",
    "site_id":"default",
    "product_id": "default",
    "product_family_id": "default",
}

exclude_features = ["fold","sold_quantity","sku","item_id","minutes_active",]

model_params = {
    'objective':'tweedie',
    'metric':'None',
    'num_iterations':1000,
    'max_bin': 63,
    'bin_construct_sample_cnt':20000000,
    'num_leaves': 2**8-1,
    'min_data_in_leaf': 2**8-1,
    'learning_rate': 0.02,
    'max_delta_step':1.,
    'feature_fraction':0.8,
    'bagging_fraction':0.8,
    'bagging_freq':1,
    'lambda_l2':0.1,
    'max_cat_threshold': 64,
    'cat_l2': 50,
    'cat_smooth': 50,
    'boost_from_average': False,
    'force_row_wise': True,
    'deterministic':True,
    'seed':SEED,
    'verbosity':-1,
}
time_features = [
    "week_day",
    "week_day_cos",
    "week_day_sin",
    "month_progress",
]
model_kwargs = {
    "model_params":model_params,
    "time_features":time_features,
    "exclude_features":exclude_features,
    "categorical_features":categorical_features,
    "ts_uid_columns":["sku",],
    #"lags": [1,],
    #"window_functions":{
    #    "mean":   (None, [1,], [3,7,]),
    #    "std":    (None, [1,], [3,7]),
    #},
}

***
## Loss tuning: tweedie

In [12]:
all_results = list()

def objective(trial):
    vp = trial.suggest_discrete_uniform("vp", 1, 1.9, 0.1)
    w = trial.suggest_discrete_uniform("w", 0., 1., 0.1)
            
    result = {
        "tweedie_variance_power":vp,
        "min_weight":w,
    }
    
    model_kwargs["model_params"]["tweedie_variance_power"] = vp
    
    if "weight" in dataset.columns:
        dataset.drop("weight", axis=1, inplace=True)
    if w < 1:
        scaler = preprocessing.MinMaxScaler(feature_range=(w,1))
        dataset["weight"] = scaler.fit_transform(dataset["minutes_active"].values.reshape(-1,1)).ravel()

    models_by_site = list()
    sites = ["MLA","MLB","MLM"]
    
    # training the models by site
    for site in sites:
        train_data = dataset.query("site_id == @site").reset_index(drop=True)
        valid_idx = train_data.query("ds > '2021-03-01'").index

        model = LightGBMForecaster(**model_kwargs)
        model.prepare_features(train_data, valid_idx)
        model.train_features = reduce_mem_usage(model.train_features, verbose=False)
        model.valid_features = reduce_mem_usage(model.valid_features, verbose=False)
        gc.collect()

        evaluator = RMSSE(train_data.loc[valid_idx, ["sku","y"]], scales)
        model.fit(fit_kwargs={"verbose_eval":False, "feval":evaluator.evaluate})
        models_by_site.append(model)

    errors = list()
    sites = ["MLA","MLB","MLM"]
    for site,model in zip(sites,models_by_site):
        error = model.model.model.best_score["valid_0"]["rmsse"]
        errors.append(error)
        
    result["rmsse_MLA"] = errors[0]
    result["rmsse_MLB"] = errors[1]
    result["rmsse_MLM"] = errors[2]
    result["rmsse_AVG"] = np.mean(errors)
    
    # generating oof predictions
    all_preds = list()
    for site,model in zip(sites,models_by_site):
        preds = model.predict(dataset.query("site_id==@site & ds>'2021-03-01'"), recursive=False)
        all_preds.append(preds)
    oof = pd.concat(all_preds, axis=0, ignore_index=True)
    
    # generates the ID predictions
    predictor = InventoryDaysPredictor(dataset.query("ds <= '2021-03-01'"))
    predictor.fit(oof)
    
    valid = (
        pd.read_csv(f"{input_path}/validation_seed2_harder.csv")
        .query("sku in @skus_assess_m1")
        .reset_index(drop=True)
    )
    
    preds = list()
    for sku,df in tqdm(valid.groupby("sku")):
        preds.append(predictor.predict(sku, df.target_stock.values[0])[0])   
    valid["days_to_stockout"] = np.asarray(preds)
    valid["days_to_stockout_disc"] = valid["days_to_stockout"].astype(int)
    
    rmse1 = np.sqrt(valid.query("inventory_days <= 31").eval("(days_to_stockout - inventory_days)**2").mean())
    mean_ae1 = valid.query("inventory_days <= 31").eval("abs(days_to_stockout - inventory_days)").mean()
    median_ae1 = valid.query("inventory_days <= 31").eval("abs(days_to_stockout - inventory_days)").median()
    
    result["rmse1"] = rmse1
    result["mean_ae1"] = mean_ae1
    result["median_ae1"] = median_ae1
    
    rmse2 = np.sqrt(valid.query("sku in @skus_assess_m2").eval("(days_to_stockout - inventory_days)**2").mean())
    mean_ae2 = valid.query("sku in @skus_assess_m2").eval("abs(days_to_stockout - inventory_days)").mean()
    median_ae2 = valid.query("sku in @skus_assess_m2").eval("abs(days_to_stockout - inventory_days)").median()
    
    result["rmse2"] = rmse2
    result["mean_ae2"] = mean_ae2
    result["median_ae2"] = median_ae2
    
    all_results.append(result)
    
    del models_by_site,predictor,valid
    gc.collect()
    
    return mean_ae1

In [13]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10000, timeout=21600, n_jobs=1) # 6-hrs

[I 2021-09-07 13:02:01,506] A new study created in memory with name: no-name-bdef6539-519f-4de9-b2d2-1c002ecbc750
100%|██████████| 482635/482635 [01:18<00:00, 6116.53it/s]
[I 2021-09-07 13:15:20,384] Trial 0 finished with value: 7.426325532003187 and parameters: {'vp': 1.7000000000000002, 'w': 0.6000000000000001}. Best is trial 0 with value: 7.426325532003187.
100%|██████████| 482635/482635 [01:14<00:00, 6486.44it/s]
[I 2021-09-07 13:28:04,697] Trial 1 finished with value: 7.343082485082378 and parameters: {'vp': 1.7000000000000002, 'w': 0.30000000000000004}. Best is trial 1 with value: 7.343082485082378.
100%|██████████| 482635/482635 [01:24<00:00, 5736.40it/s]
[I 2021-09-07 13:42:47,170] Trial 2 finished with value: 7.566563663864481 and parameters: {'vp': 1.7000000000000002, 'w': 1.0}. Best is trial 1 with value: 7.343082485082378.
100%|██████████| 482635/482635 [01:14<00:00, 6475.94it/s]
[I 2021-09-07 13:57:09,464] Trial 3 finished with value: 7.326574546562435 and parameters: {'vp

In [15]:
study.trials_dataframe().drop_duplicates(subset=["params_vp","params_w"]).sort_values("value").head(20)

,number,value,datetime_start,datetime_complete,duration,params_vp,params_w,state
7,7,7.185425,2021-09-07 14:41:26.272422,2021-09-07 14:54:54.340113,0 days 00:13:28.067691,1.0,0.0,COMPLETE
13,13,7.200398,2021-09-07 15:59:51.365013,2021-09-07 16:11:42.987942,0 days 00:11:51.622929,1.1,0.0,COMPLETE
12,12,7.205433,2021-09-07 15:45:45.539958,2021-09-07 15:59:51.363593,0 days 00:14:05.823635,1.0,0.2,COMPLETE
19,19,7.211665,2021-09-07 17:16:46.360468,2021-09-07 17:28:47.376124,0 days 00:12:01.015656,1.1,0.1,COMPLETE
27,27,7.213497,2021-09-07 18:59:10.235052,2021-09-07 19:11:41.256587,0 days 00:12:31.021535,1.1,0.2,COMPLETE
22,22,7.229780,2021-09-07 17:55:40.936103,2021-09-07 18:07:57.229240,0 days 00:12:16.293137,1.2,0.1,COMPLETE
25,25,7.234880,2021-09-07 18:33:03.098900,2021-09-07 18:47:27.141239,0 days 00:14:24.042339,1.0,0.3,COMPLETE
26,26,7.234931,2021-09-07 18:47:27.143089,2021-09-07 18:59:10.233571,0 days 00:11:43.090482,1.3,0.0,COMPLETE
14,14,7.236334,2021-09-07 16:11:42.989782,2021-09-07 16:25:41.854791,0 days 00:13:58.865009,1.1,0.3,COMPLETE
17,17,7.238018,2021-09-07 16:49:48.337573,2021-09-07 17:02:36.793392,0 days 00:12:48.455819,1.2,0.2,COMPLETE


In [18]:
result_df = pd.DataFrame(all_results).drop_duplicates(subset=["tweedie_variance_power","min_weight"])

In [21]:
result_df.sort_values("rmsse_AVG").head(10)

,tweedie_variance_power,min_weight,rmsse_MLA,rmsse_MLB,rmsse_MLM,rmsse_AVG,rmse1,mean_ae1,median_ae1,rmse2,mean_ae2,median_ae2
2,1.7,1.0,0.805258,0.857530,0.822245,0.828344,9.990331,7.566564,5.829331,9.098441,6.890774,5.219919
8,1.3,1.0,0.806388,0.858401,0.825152,0.829980,9.917013,7.478044,5.675513,9.027413,6.810459,5.098709
5,1.4,0.8,0.809836,0.861938,0.829784,0.833853,9.883068,7.440252,5.619990,9.008331,6.787627,5.063277
6,1.2,0.8,0.810161,0.862606,0.831511,0.834759,9.857319,7.405573,5.551460,8.985001,6.757474,5.013705
20,1.5,0.7,0.811771,0.863907,0.832079,0.835919,9.880886,7.435148,5.607257,9.013592,6.789258,5.064686
0,1.7,0.6,0.813356,0.865905,0.833603,0.837621,9.860928,7.426326,5.600523,9.014629,6.798086,5.080957
4,1.7,0.5,0.816144,0.868990,0.837491,0.840875,9.831950,7.393832,5.545684,9.001306,6.781950,5.055488
3,1.3,0.5,0.818708,0.870878,0.842170,0.843919,9.780895,7.326575,5.440409,8.948242,6.717068,5.000000
18,1.4,0.4,0.821713,0.874348,0.846002,0.847354,9.762961,7.310208,5.414025,8.950784,6.720038,5.000000
1,1.7,0.3,0.823470,0.877096,0.847503,0.849356,9.789844,7.343082,5.462503,8.995235,6.766642,5.022541


In [22]:
result_df.sort_values("rmse1").head(10)

,tweedie_variance_power,min_weight,rmsse_MLA,rmsse_MLB,rmsse_MLM,rmsse_AVG,rmse1,mean_ae1,median_ae1,rmse2,mean_ae2,median_ae2
12,1.0,0.2,0.835438,0.889104,0.866876,0.863806,9.685762,7.205433,5.230327,8.921350,6.664996,4.996067
7,1.0,0.0,0.853327,0.912556,0.892977,0.886287,9.689780,7.185425,5.170047,8.986487,6.707099,4.999936
27,1.1,0.2,0.834489,0.888115,0.864996,0.862534,9.690730,7.213497,5.249313,8.924925,6.672050,5.000000
19,1.1,0.1,0.842092,0.897391,0.876072,0.871852,9.700767,7.211665,5.224567,8.958040,6.692327,5.000000
13,1.1,0.0,0.851529,0.910970,0.890152,0.884217,9.701710,7.200398,5.186500,8.996739,6.719714,5.000000
14,1.1,0.3,0.828611,0.881404,0.856690,0.855569,9.706529,7.236334,5.292914,8.917322,6.671595,5.000000
25,1.0,0.3,0.829038,0.882220,0.858241,0.856500,9.710643,7.234880,5.280963,8.921845,6.670782,5.000000
17,1.2,0.2,0.833389,0.887214,0.863265,0.861289,9.713518,7.238018,5.287325,8.943669,6.691602,5.000000
22,1.2,0.1,0.841118,0.896333,0.873923,0.870458,9.715115,7.229780,5.252754,8.972831,6.710128,5.000000
26,1.3,0.0,0.848384,0.907878,0.884496,0.880253,9.727675,7.234931,5.245163,9.018176,6.747876,5.000000


In [23]:
result_df.sort_values("mean_ae1").head(10)

,tweedie_variance_power,min_weight,rmsse_MLA,rmsse_MLB,rmsse_MLM,rmsse_AVG,rmse1,mean_ae1,median_ae1,rmse2,mean_ae2,median_ae2
7,1.0,0.0,0.853327,0.912556,0.892977,0.886287,9.689780,7.185425,5.170047,8.986487,6.707099,4.999936
13,1.1,0.0,0.851529,0.910970,0.890152,0.884217,9.701710,7.200398,5.186500,8.996739,6.719714,5.000000
12,1.0,0.2,0.835438,0.889104,0.866876,0.863806,9.685762,7.205433,5.230327,8.921350,6.664996,4.996067
19,1.1,0.1,0.842092,0.897391,0.876072,0.871852,9.700767,7.211665,5.224567,8.958040,6.692327,5.000000
27,1.1,0.2,0.834489,0.888115,0.864996,0.862534,9.690730,7.213497,5.249313,8.924925,6.672050,5.000000
22,1.2,0.1,0.841118,0.896333,0.873923,0.870458,9.715115,7.229780,5.252754,8.972831,6.710128,5.000000
25,1.0,0.3,0.829038,0.882220,0.858241,0.856500,9.710643,7.234880,5.280963,8.921845,6.670782,5.000000
26,1.3,0.0,0.848384,0.907878,0.884496,0.880253,9.727675,7.234931,5.245163,9.018176,6.747876,5.000000
14,1.1,0.3,0.828611,0.881404,0.856690,0.855569,9.706529,7.236334,5.292914,8.917322,6.671595,5.000000
17,1.2,0.2,0.833389,0.887214,0.863265,0.861289,9.713518,7.238018,5.287325,8.943669,6.691602,5.000000


In [24]:
result_df.sort_values("median_ae1").head(10)

,tweedie_variance_power,min_weight,rmsse_MLA,rmsse_MLB,rmsse_MLM,rmsse_AVG,rmse1,mean_ae1,median_ae1,rmse2,mean_ae2,median_ae2
7,1.0,0.0,0.853327,0.912556,0.892977,0.886287,9.689780,7.185425,5.170047,8.986487,6.707099,4.999936
13,1.1,0.0,0.851529,0.910970,0.890152,0.884217,9.701710,7.200398,5.186500,8.996739,6.719714,5.000000
19,1.1,0.1,0.842092,0.897391,0.876072,0.871852,9.700767,7.211665,5.224567,8.958040,6.692327,5.000000
12,1.0,0.2,0.835438,0.889104,0.866876,0.863806,9.685762,7.205433,5.230327,8.921350,6.664996,4.996067
26,1.3,0.0,0.848384,0.907878,0.884496,0.880253,9.727675,7.234931,5.245163,9.018176,6.747876,5.000000
27,1.1,0.2,0.834489,0.888115,0.864996,0.862534,9.690730,7.213497,5.249313,8.924925,6.672050,5.000000
22,1.2,0.1,0.841118,0.896333,0.873923,0.870458,9.715115,7.229780,5.252754,8.972831,6.710128,5.000000
25,1.0,0.3,0.829038,0.882220,0.858241,0.856500,9.710643,7.234880,5.280963,8.921845,6.670782,5.000000
17,1.2,0.2,0.833389,0.887214,0.863265,0.861289,9.713518,7.238018,5.287325,8.943669,6.691602,5.000000
14,1.1,0.3,0.828611,0.881404,0.856690,0.855569,9.706529,7.236334,5.292914,8.917322,6.671595,5.000000


In [25]:
result_df.sort_values("rmse2").head(10)

,tweedie_variance_power,min_weight,rmsse_MLA,rmsse_MLB,rmsse_MLM,rmsse_AVG,rmse1,mean_ae1,median_ae1,rmse2,mean_ae2,median_ae2
14,1.1,0.3,0.828611,0.881404,0.856690,0.855569,9.706529,7.236334,5.292914,8.917322,6.671595,5.000000
12,1.0,0.2,0.835438,0.889104,0.866876,0.863806,9.685762,7.205433,5.230327,8.921350,6.664996,4.996067
25,1.0,0.3,0.829038,0.882220,0.858241,0.856500,9.710643,7.234880,5.280963,8.921845,6.670782,5.000000
27,1.1,0.2,0.834489,0.888115,0.864996,0.862534,9.690730,7.213497,5.249313,8.924925,6.672050,5.000000
17,1.2,0.2,0.833389,0.887214,0.863265,0.861289,9.713518,7.238018,5.287325,8.943669,6.691602,5.000000
3,1.3,0.5,0.818708,0.870878,0.842170,0.843919,9.780895,7.326575,5.440409,8.948242,6.717068,5.000000
18,1.4,0.4,0.821713,0.874348,0.846002,0.847354,9.762961,7.310208,5.414025,8.950784,6.720038,5.000000
19,1.1,0.1,0.842092,0.897391,0.876072,0.871852,9.700767,7.211665,5.224567,8.958040,6.692327,5.000000
22,1.2,0.1,0.841118,0.896333,0.873923,0.870458,9.715115,7.229780,5.252754,8.972831,6.710128,5.000000
6,1.2,0.8,0.810161,0.862606,0.831511,0.834759,9.857319,7.405573,5.551460,8.985001,6.757474,5.013705


In [26]:
result_df.sort_values("mean_ae2").head(10)

,tweedie_variance_power,min_weight,rmsse_MLA,rmsse_MLB,rmsse_MLM,rmsse_AVG,rmse1,mean_ae1,median_ae1,rmse2,mean_ae2,median_ae2
12,1.0,0.2,0.835438,0.889104,0.866876,0.863806,9.685762,7.205433,5.230327,8.921350,6.664996,4.996067
25,1.0,0.3,0.829038,0.882220,0.858241,0.856500,9.710643,7.234880,5.280963,8.921845,6.670782,5.000000
14,1.1,0.3,0.828611,0.881404,0.856690,0.855569,9.706529,7.236334,5.292914,8.917322,6.671595,5.000000
27,1.1,0.2,0.834489,0.888115,0.864996,0.862534,9.690730,7.213497,5.249313,8.924925,6.672050,5.000000
17,1.2,0.2,0.833389,0.887214,0.863265,0.861289,9.713518,7.238018,5.287325,8.943669,6.691602,5.000000
19,1.1,0.1,0.842092,0.897391,0.876072,0.871852,9.700767,7.211665,5.224567,8.958040,6.692327,5.000000
7,1.0,0.0,0.853327,0.912556,0.892977,0.886287,9.689780,7.185425,5.170047,8.986487,6.707099,4.999936
22,1.2,0.1,0.841118,0.896333,0.873923,0.870458,9.715115,7.229780,5.252754,8.972831,6.710128,5.000000
3,1.3,0.5,0.818708,0.870878,0.842170,0.843919,9.780895,7.326575,5.440409,8.948242,6.717068,5.000000
13,1.1,0.0,0.851529,0.910970,0.890152,0.884217,9.701710,7.200398,5.186500,8.996739,6.719714,5.000000


In [27]:
result_df.sort_values("median_ae2").head(10)

,tweedie_variance_power,min_weight,rmsse_MLA,rmsse_MLB,rmsse_MLM,rmsse_AVG,rmse1,mean_ae1,median_ae1,rmse2,mean_ae2,median_ae2
12,1.0,0.2,0.835438,0.889104,0.866876,0.863806,9.685762,7.205433,5.230327,8.921350,6.664996,4.996067
7,1.0,0.0,0.853327,0.912556,0.892977,0.886287,9.689780,7.185425,5.170047,8.986487,6.707099,4.999936
25,1.0,0.3,0.829038,0.882220,0.858241,0.856500,9.710643,7.234880,5.280963,8.921845,6.670782,5.000000
22,1.2,0.1,0.841118,0.896333,0.873923,0.870458,9.715115,7.229780,5.252754,8.972831,6.710128,5.000000
19,1.1,0.1,0.842092,0.897391,0.876072,0.871852,9.700767,7.211665,5.224567,8.958040,6.692327,5.000000
18,1.4,0.4,0.821713,0.874348,0.846002,0.847354,9.762961,7.310208,5.414025,8.950784,6.720038,5.000000
17,1.2,0.2,0.833389,0.887214,0.863265,0.861289,9.713518,7.238018,5.287325,8.943669,6.691602,5.000000
14,1.1,0.3,0.828611,0.881404,0.856690,0.855569,9.706529,7.236334,5.292914,8.917322,6.671595,5.000000
13,1.1,0.0,0.851529,0.910970,0.890152,0.884217,9.701710,7.200398,5.186500,8.996739,6.719714,5.000000
26,1.3,0.0,0.848384,0.907878,0.884496,0.880253,9.727675,7.234931,5.245163,9.018176,6.747876,5.000000


***